In [14]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import NoSuchElementException, ElementClickInterceptedException
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException
from selenium.webdriver.common.alert import Alert
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.action_chains import ActionChains
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import time
import os
import random
import math
import pyautogui
import pyperclip
import tkinter as tk
from tkinter import messagebox, filedialog

date = time.strftime("%Y%m%d")

# 키워드 목록 정의
keywords = [
    "주일 예배 대표 기도문 100개",
    "5 월 주일 대표 기도문",
    "5월 셋째 주일 대표 기도문",
    "5 월 넷째 주 대표 기도문 모음",
    "6월 첫째 주일 대표 기도문",
    "6월 둘째 주 대표 기도문 모음 (2025년)",
    "6월 셋째 주일 대표 기도문 모음 (2025년)"
]

font_path='/Users/a/Desktop/font/나눔 글꼴/나눔고딕/NanumFontSetup_TTF_GOTHIC/NanumGothicExtraBold.ttf'
image_final_folder = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated"

# 버튼 이미지 변수 정의
check_emoji =""
hand_emoji=" 바로가기 ▶"

def generate_banner_image_with_text(text, font_path, banner_save_folder, filename):
    # 정방형 이미지로 변경 (1000x1000)
    width, height = 1000, 1000
    font_size = 120  # 정방형에 맞게 폰트 크기 조정

    # 폰트 로드
    font = ImageFont.truetype(font_path, size=font_size)
    
    # 채도가 낮춰진 그라데이션 색상 조합
    soft_gradients = [
        [(220, 150, 150), (255, 255, 255)],  # 부드러운 분홍 → 흰색
        [(150, 180, 230), (255, 255, 255)],  # 부드러운 파랑 → 흰색
        [(230, 200, 150), (255, 255, 255)],  # 부드러운 주황 → 흰색
        [(170, 215, 170), (255, 255, 255)],  # 부드러운 연두 → 흰색
        [(190, 170, 220), (255, 255, 255)],  # 부드러운 보라 → 흰색
        [(220, 170, 200), (255, 255, 255)],  # 부드러운 핑크 → 흰색
        [(150, 190, 200), (255, 255, 255)],  # 부드러운 옥색 → 흰색
    ]
    
    # 랜덤한 그라데이션 색상 선택
    gradient_colors = random.choice(soft_gradients)
    start_color, end_color = gradient_colors
    
    # 빈 이미지 생성
    image = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(image)
    
    # 랜덤 그라데이션 방향 선택
    direction = random.choice(['left_to_right', 'right_to_left', 'top_to_bottom', 'bottom_to_top', 'diagonal_tl', 'diagonal_tr'])
    
    # 그라데이션 생성 (다양한 방향)
    for y in range(height):
        for x in range(width):
            # 방향에 따른 비율 계산 (0.0 ~ 1.0)
            if direction == 'left_to_right':
                ratio = x / width
            elif direction == 'right_to_left':
                ratio = 1 - (x / width)
            elif direction == 'top_to_bottom':
                ratio = y / height
            elif direction == 'bottom_to_top':
                ratio = 1 - (y / height)
            elif direction == 'diagonal_tl':  # 왼쪽 상단에서 오른쪽 하단
                ratio = (x + y) / (width + height)
            else:  # diagonal_tr: 오른쪽 상단에서 왼쪽 하단
                ratio = (width - x + y) / (width + height)
            
            # 부드러운 변화를 위해 지수 조정 (1.3으로 낮춤)
            ratio = pow(ratio, 1.3)  # 더 부드러운 전환을 위해 지수 감소
            
            # 색상 보간
            r = int(start_color[0] * (1 - ratio) + end_color[0] * ratio)
            g = int(start_color[1] * (1 - ratio) + end_color[1] * ratio)
            b = int(start_color[2] * (1 - ratio) + end_color[2] * ratio)
            
            # 픽셀 색상 설정
            image.putpixel((x, y), (r, g, b))
    
    # 텍스트를 3단어씩 줄바꿈
    words = text.split()
    lines = []
    
    # 3단어씩 그룹화
    for i in range(0, len(words), 3):
        line = ' '.join(words[i:i+3])
        lines.append(line)
    
    # 줄바꿈된 텍스트
    formatted_text = '\n'.join(lines)
    
    # 텍스트 박스 계산
    textbbox = draw.multiline_textbbox((0, 0), formatted_text, font=font, spacing=10)
    
    # 텍스트 그리기 위치 계산
    text_x = (width - textbbox[2] - textbbox[0]) / 2
    text_y = (height - textbbox[3] - textbbox[1]) / 2

    # 더 진한 회색 색상 정의 (더 선명하게)
    dark_gray_color = (30, 30, 30)  # 더 진한 회색
    
    # 텍스트에 테두리 효과 추가 (더 선명하게 보이도록)
    # 텍스트 그림자 효과 (약간의 오프셋으로 그림자 추가)
    offset = 3
    shadow_color = (220, 220, 220)
    draw.multiline_text((text_x+offset, text_y+offset), formatted_text, font=font, fill=shadow_color, align="center", spacing=80)

    # 텍스트 그리기
    draw.multiline_text((text_x, text_y), formatted_text, font=font, fill=(dark_gray_color), align="center", spacing=80)

    # 선명도 향상 (적당히)
    image = image.filter(ImageFilter.SHARPEN)
    
    # 이미지 파일명에 그라데이션 방향 정보 추가
    save_path = os.path.join(banner_save_folder, f"{filename}")
    image.save(save_path, quality=95, dpi=(300, 300))
    print(f"생성 완료: {save_path}")
    return save_path

# 폴더 생성
os.makedirs(image_final_folder, exist_ok=True)

# 각 키워드에 대해 이미지 1개씩 생성
for keyword in keywords:
    # 키워드별 폴더 생성
    keyword_folder = os.path.join(image_final_folder, keyword)
    os.makedirs(keyword_folder, exist_ok=True)
    
    # 배너 이미지 생성 (각 키워드당 1개만)
    filename = f"banner_{keyword}.png"
    generate_banner_image_with_text(keyword, font_path, keyword_folder, filename)

print("모든 키워드에 대한 이미지 생성 완료")

생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/주일 예배 대표 기도문 100개/banner_주일 예배 대표 기도문 100개.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/5 월 주일 대표 기도문/banner_5 월 주일 대표 기도문.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/5월 셋째 주일 대표 기도문/banner_5월 셋째 주일 대표 기도문.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/5 월 넷째 주 대표 기도문 모음/banner_5 월 넷째 주 대표 기도문 모음.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/6월 첫째 주일 대표 기도문/banner_6월 첫째 주일 대표 기도문.png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/6월 둘째 주 대표 기도문 모음 (2025년)/banner_6월 둘째 주 대표 기도문 모음 (2025년).png
생성 완료: /Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated/6월 셋째 주일 대표 기도문 모음 (2025년)/banner_6월 셋째 주일 대표 기도문 모음 (2025년).png
모든 키워드에 대한 이미지 생성 완료


In [12]:
#####그라데이션 최종 - 6개 이미지, 1개 키워드

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time
import pyperclip
import pyautogui
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog
from PIL import Image, ImageTk
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException
import math
import pyautogui
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import re
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import os
import random
import math
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from PIL import Image, ImageDraw, ImageFont
import os
import random
import math

date = time.strftime("%Y%m%d")

# 변경된 부분: 키워드 직접 정의 및 저장 경로 변경
keyword = "주일 예배 대표 기도문 100개"
font_path='/Users/a/Desktop/font/나눔 글꼴/나눔고딕/NanumFontSetup_TTF_GOTHIC/NanumGothicExtraBold.ttf'
image_final_folder = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated"

#버튼 이미지 변수 정의
check_emoji =""
hand_emoji=" 바로가기 ▶"

# 그라데이션 색상 팔레트 (부드러운 색상 조합들)
gradient_colors = [
    [(255, 97, 97), (255, 255, 255)],      # 오렌지-핑크 그라데이션
    [(65, 105, 225), (255, 255, 255)],     # 블루-퍼플 그라데이션
    [(30, 144, 255), (255, 255, 255)],      # 딥스카이블루-터키oise
    [(255, 105, 180), (255, 255, 255)],    # 핑크-핫핑크
    [(255, 179, 71), (255, 255, 255)],    # 골드-살몬
    [(100, 149, 237), (255, 255, 255)],   # 코렌플라워블루-라이트스틸블루
    [(255, 20, 147), (1255, 255, 255)],     # 딥핑크-미디엄 바이올렛 레드
]


def generate_banner_image_with_text(text, font_path, banner_save_folder, filename):
    # 정방형 이미지로 변경 (1000x1000)
    width, height = 1000, 1000
    font_size = 120  # 정방형에 맞게 폰트 크기 조정

    # 폰트 로드
    font = ImageFont.truetype(font_path, size=font_size)
    
    # 채도가 낮춰진 그라데이션 색상 조합
    soft_gradients = [
        [(220, 150, 150), (255, 255, 255)],  # 부드러운 분홍 → 흰색
        [(150, 180, 230), (255, 255, 255)],  # 부드러운 파랑 → 흰색
        [(230, 200, 150), (255, 255, 255)],  # 부드러운 주황 → 흰색
        [(170, 215, 170), (255, 255, 255)],  # 부드러운 연두 → 흰색
        [(190, 170, 220), (255, 255, 255)],  # 부드러운 보라 → 흰색
        [(220, 170, 200), (255, 255, 255)],  # 부드러운 핑크 → 흰색
        [(150, 190, 200), (255, 255, 255)],  # 부드러운 옥색 → 흰색
    ]
    
    # 랜덤한 그라데이션 색상 선택
    gradient_colors = random.choice(soft_gradients)
    start_color, end_color = gradient_colors
    
    # 빈 이미지 생성
    image = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(image)
    
    # 랜덤 그라데이션 방향 선택
    direction = random.choice(['left_to_right', 'right_to_left', 'top_to_bottom', 'bottom_to_top', 'diagonal_tl', 'diagonal_tr'])
    
    # 그라데이션 생성 (다양한 방향)
    for y in range(height):
        for x in range(width):
            # 방향에 따른 비율 계산 (0.0 ~ 1.0)
            if direction == 'left_to_right':
                ratio = x / width
            elif direction == 'right_to_left':
                ratio = 1 - (x / width)
            elif direction == 'top_to_bottom':
                ratio = y / height
            elif direction == 'bottom_to_top':
                ratio = 1 - (y / height)
            elif direction == 'diagonal_tl':  # 왼쪽 상단에서 오른쪽 하단
                ratio = (x + y) / (width + height)
            else:  # diagonal_tr: 오른쪽 상단에서 왼쪽 하단
                ratio = (width - x + y) / (width + height)
            
            # 부드러운 변화를 위해 지수 조정 (1.3으로 낮춤)
            ratio = pow(ratio, 1.3)  # 더 부드러운 전환을 위해 지수 감소
            
            # 색상 보간
            r = int(start_color[0] * (1 - ratio) + end_color[0] * ratio)
            g = int(start_color[1] * (1 - ratio) + end_color[1] * ratio)
            b = int(start_color[2] * (1 - ratio) + end_color[2] * ratio)
            
            # 픽셀 색상 설정
            image.putpixel((x, y), (r, g, b))
    
    # 텍스트를 3단어씩 줄바꿈
    words = text.split()
    lines = []
    
    # 3단어씩 그룹화
    for i in range(0, len(words), 3):
        line = ' '.join(words[i:i+3])
        lines.append(line)
    
    # 줄바꿈된 텍스트
    formatted_text = '\n'.join(lines)
    
    # 텍스트 박스 계산
    textbbox = draw.multiline_textbbox((0, 0), formatted_text, font=font, spacing=10)
    
    # 텍스트 그리기 위치 계산
    text_x = (width - textbbox[2] - textbbox[0]) / 2
    text_y = (height - textbbox[3] - textbbox[1]) / 2

    # 더 진한 회색 색상 정의 (더 선명하게)
    dark_gray_color = (30, 30, 30)  # 더 진한 회색
    
    # 텍스트에 테두리 효과 추가 (더 선명하게 보이도록)
    # 텍스트 그림자 효과 (약간의 오프셋으로 그림자 추가)
    offset = 3
    shadow_color = (220, 220, 220)
    draw.multiline_text((text_x+offset, text_y+offset), formatted_text, font=font, fill=shadow_color, align="center", spacing=80)

    # 텍스트 그리기
    draw.multiline_text((text_x, text_y), formatted_text, font=font, fill=(dark_gray_color), align="center", spacing=80)

    # 선명도 향상 (적당히)
    image = image.filter(ImageFilter.SHARPEN)
    
    # 이미지 파일명에 그라데이션 방향 정보 추가
    save_path = os.path.join(banner_save_folder, f"{filename.split('.')[0]}_{direction}.png")
    image.save(save_path, quality=95, dpi=(300, 300))
# 변경된 부분: 엑셀 파일 대신 단일 키워드 사용

# 폴더 생성
os.makedirs(image_final_folder, exist_ok=True)
keyword_folder = os.path.join(image_final_folder, keyword)
os.makedirs(keyword_folder, exist_ok=True)

# 배너 이미지 생성
for i in range(1, 8):
    filename = f"banner_{keyword}_{i}.png"
    generate_banner_image_with_text(keyword, font_path, keyword_folder, filename)


print("생성 완료")

생성 완료


In [ ]:
######변수 정의 ###############

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time
import pyperclip
import pyautogui
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog
from PIL import Image, ImageTk
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException
import math
import pyautogui
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import re
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import os
import random
import math
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from PIL import Image, ImageDraw, ImageFont
import os
import random
import math

date = time.strftime("%Y%m%d")

# 변경된 부분: 키워드 직접 정의 및 저장 경로 변경
keyword = "청년 주일 대표 기도문 모음"
font_path='/Users/a/Desktop/font/나눔 글꼴/나눔고딕/NanumFontSetup_TTF_GOTHIC/NanumGothicExtraBold.ttf'
image_final_folder = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated"

#버튼 이미지 변수 정의
check_emoji =""
hand_emoji=" 바로가기 ▶"

# 그라데이션 색상 팔레트 (부드러운 색상 조합들)
gradient_colors = [
    [(255, 97, 97), (255, 255, 255)],      # 오렌지-핑크 그라데이션
    [(65, 105, 225), (255, 255, 255)],     # 블루-퍼플 그라데이션
    [(30, 144, 255), (255, 255, 255)],      # 딥스카이블루-터키oise
    [(255, 105, 180), (255, 255, 255)],    # 핑크-핫핑크
    [(255, 179, 71), (255, 255, 255)],    # 골드-살몬
    [(100, 149, 237), (255, 255, 255)],   # 코렌플라워블루-라이트스틸블루
    [(255, 20, 147), (1255, 255, 255)],     # 딥핑크-미디엄 바이올렛 레드
]


def generate_banner_image_with_text(text, font_path, banner_save_folder, filename):
    # 정방형 이미지로 변경 (1000x1000)
    width, height = 1000, 1000
    font_size = 120  # 정방형에 맞게 폰트 크기 조정

    # 폰트 로드
    font = ImageFont.truetype(font_path, size=font_size)
    
    # 그라데이션용 파스텔 색상 조합
    pastel_gradients = [
        [(255, 255, 230), (255, 255, 255)],  # 매우 연한 노랑 → 연한 분홍
        [(255, 250, 230), (255, 255, 255)],  # 매우 연한 크림 → 연한 라벤더
        [(255, 245, 230), (255, 255, 255)],  # 매우 연한 살구 → 연한 민트
        [(255, 240, 240), (255, 255, 255)],  # 매우 연한 분홍 → 연한 하늘
        [(255, 235, 235), (255, 255, 255)],  # 매우 연한 빨강 → 연한 그린
        [(255, 230, 230), (255, 255, 255)],  # 매우 연한 분홍 → 연한 블루
        [(255, 240, 225), (255, 255, 255)],  # 매우 연한 주황 → 연한 라임
    ]
    
    # 랜덤한 그라데이션 색상 선택
    gradient_colors = random.choice(pastel_gradients)
    start_color, end_color = gradient_colors
    
    # 빈 이미지 생성
    image = Image.new("RGB", (width, height))
    draw = ImageDraw.Draw(image)
    
    # 그라데이션 생성 (대각선 방향)
    for y in range(height):
        for x in range(width):
            # 대각선 위치에 따른 비율 계산 (0.0 ~ 1.0)
            ratio = (x + y) / (width + height)
            
            # 색상 보간
            r = int(start_color[0] * (1 - ratio) + end_color[0] * ratio)
            g = int(start_color[1] * (1 - ratio) + end_color[1] * ratio)
            b = int(start_color[2] * (1 - ratio) + end_color[2] * ratio)
            
            # 픽셀 색상 설정
            image.putpixel((x, y), (r, g, b))
    
    # 텍스트를 3단어씩 줄바꿈
    words = text.split()
    lines = []
    
    # 3단어씩 그룹화
    for i in range(0, len(words), 3):
        line = ' '.join(words[i:i+3])
        lines.append(line)
    
    # 줄바꿈된 텍스트
    formatted_text = '\n'.join(lines)
    
    # 텍스트 박스 계산
    textbbox = draw.multiline_textbbox((0, 0), formatted_text, font=font, spacing=10)
    
    # 텍스트 그리기 위치 계산
    text_x = (width - textbbox[2] - textbbox[0]) / 2
    text_y = (height - textbbox[3] - textbbox[1]) / 2

    # 진한 회색 색상 정의
    dark_gray_color = (51, 51, 51)  # 아주 진한 회색 (거의 검정에 가까운 진한 회색)

    # 텍스트 그리기
    draw.multiline_text((text_x, text_y), formatted_text, font=font, fill=(dark_gray_color), align="center", spacing=80)

    # 선명도 향상
    image = image.filter(ImageFilter.SHARPEN)

    # 이미지 저장 (높은 품질)
    save_path = os.path.join(banner_save_folder, filename)
    image.save(save_path, quality=95, dpi=(300, 300))
# 메인 작업 로직
# 변경된 부분: 엑셀 파일 대신 단일 키워드 사용

# 폴더 생성
os.makedirs(image_final_folder, exist_ok=True)
keyword_folder = os.path.join(image_final_folder, keyword)
os.makedirs(keyword_folder, exist_ok=True)

# 배너 이미지 생성
for i in range(1, 8):
    filename = f"banner_{keyword}_{i}.png"
    generate_banner_image_with_text(keyword, font_path, keyword_folder, filename)


print("생성 완료")

In [5]:
######변수 정의 ###############

from selenium import webdriver
from selenium.webdriver.common.keys import Keys
import time
from selenium import webdriver
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.by import By
import pandas as pd
import time
import pyperclip
import pyautogui
import tkinter as tk
from tkinter import messagebox
from tkinter import filedialog
from PIL import Image, ImageTk
from selenium.common.exceptions import NoSuchElementException
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import ElementClickInterceptedException
from selenium.webdriver.common.alert import Alert
from selenium.common.exceptions import NoAlertPresentException
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
import time
from selenium.common.exceptions import UnexpectedAlertPresentException, NoAlertPresentException
import math
import pyautogui
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.common.by import By
import re
from PIL import Image, ImageDraw, ImageFont, ImageFilter
import os
import random
import math
from selenium.webdriver.chrome.service import Service
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from PIL import Image, ImageDraw, ImageFont
import os
import random
import math

date = time.strftime("%Y%m%d")

# 변경된 부분: 키워드 직접 정의 및 저장 경로 변경
keyword = "청년 주일 대표 기도문 모음"
font_path='/Users/a/Desktop/font/나눔 글꼴/나눔고딕/NanumFontSetup_TTF_GOTHIC/NanumGothicExtraBold.ttf'
image_final_folder = "/Users/a/Desktop/Work/Blog/2024 Adsense/wp/images/generated"

#버튼 이미지 변수 정의
check_emoji =""
hand_emoji=" 바로가기 ▶"

# 그라데이션 색상 팔레트 (부드러운 색상 조합들)
gradient_colors = [
    [(255, 97, 97), (255, 158, 83)],      # 오렌지-핑크 그라데이션
    [(65, 105, 225), (106, 13, 173)],     # 블루-퍼플 그라데이션
    [(30, 144, 255), (0, 206, 209)],      # 딥스카이블루-터키oise
    [(255, 105, 180), (255, 64, 129)],    # 핑크-핫핑크
    [(255, 179, 71), (255, 102, 102)],    # 골드-살몬
    [(100, 149, 237), (176, 196, 222)],   # 코렌플라워블루-라이트스틸블루
    [(255, 20, 147), (199, 21, 133)],     # 딥핑크-미디엄 바이올렛 레드
]


def generate_banner_image_with_text(text, font_path, banner_save_folder, filename):
    # 정방형 이미지로 변경 (1000x1000)
    width, height = 1000, 1000
    font_size = 120  # 정방형에 맞게 폰트 크기 조정

    # 폰트 로드
    font = ImageFont.truetype(font_path, size=font_size)
    
    # 연한 노랑과 연한 빨강 계열의 색상 목록
    # 훨씬 더 연한 파스텔 색상들
    light_colors = [
        (255, 255, 230),  # 매우 연한 노랑
        (255, 250, 230),  # 매우 연한 크림색
        (255, 245, 230),  # 매우 연한 살구색
        (255, 240, 240),  # 매우 연한 분홍
        (255, 235, 235),  # 매우 연한 빨강
        (255, 230, 230),  # 매우 연한 분홍/빨강
        (255, 240, 225),  # 매우 연한 주황
    ]

    # 랜덤한 배경 색상 생성
    bg_color = random.choice(light_colors)

    # 텍스트를 적용할 이미지 생성
    image = Image.new("RGB", (width, height), color=bg_color)
    draw = ImageDraw.Draw(image)

    # 작은 도형 추가 함수
    def draw_small_shapes(draw):
        shapes = ["circle", "square", "triangle"]
        for _ in range(15):  # 도형 개수 늘림
            shape = random.choice(shapes)
            center_x = random.randint(0, width)
            center_y = random.randint(0, height)
            shape_color = (random.randint(0, 255), random.randint(0, 255), random.randint(0, 255), 100)  # 약간 투명하게

            if shape == "circle":
                radius = random.randint(10, 40)
                draw.ellipse([(center_x - radius, center_y - radius), (center_x + radius, center_y + radius)], fill=shape_color)
            elif shape == "square":
                side_length = random.randint(20, 60)
                draw.rectangle([(center_x - side_length, center_y - side_length), (center_x + side_length, center_y + side_length)], fill=shape_color)
            elif shape == "triangle":
                size = random.randint(20, 60)
                triangle = [
                    (center_x, center_y - size),
                    (center_x + (math.sqrt(3) / 2) * size, center_y + size / 2),
                    (center_x - (math.sqrt(3) / 2) * size, center_y + size / 2)
                ]
                draw.polygon(triangle, fill=shape_color)

    # 작은 도형 추가 안함
    #draw_small_shapes(draw)
    
    # 텍스트를 3단어씩 줄바꿈
    words = text.split()
    lines = []
    
    # 3단어씩 그룹화
    for i in range(0, len(words), 3):
        line = ' '.join(words[i:i+3])
        lines.append(line)
    
    # 줄바꿈된 텍스트
    formatted_text = '\n'.join(lines)
    
    # 텍스트 박스 계산
    textbbox = draw.multiline_textbbox((0, 0), formatted_text, font=font, spacing=10)
    
    # 텍스트 그리기 위치 계산
    text_x = (width - textbbox[2] - textbbox[0]) / 2
    text_y = (height - textbbox[3] - textbbox[1]) / 2

    # 그림자 효과 추가 안할래
    #shadow_offset = 4
    #shadow_color = (0, 0, 0, 100)  # 투명한 검은색 그림자
    #draw.multiline_text((text_x+shadow_offset, text_y+shadow_offset), formatted_text, font=font, fill=shadow_color, align="center", spacing=30)

    # 텍스트 그리기
    # 진한 회색 색상 정의
    dark_gray_color = (51, 51, 51)  # 아주 진한 회색 (거의 검정에 가까운 진한 회색)

    draw.multiline_text((text_x, text_y), formatted_text, font=font, fill=(dark_gray_color), align="center", spacing=80)

    # 텍스트 박스 계산 (줄 위치 계산용)
    text_width = textbbox[2] - textbbox[0]
    
    # 텍스트 아래에 라인 추가 안함요
    #line_y = text_y + textbbox[3] + 20
    #draw.line([(width//2 - text_width//2, line_y), (width//2 + text_width//2, line_y)], fill=(255, 255, 255), width=4)

    # 선명도 향상
    image = image.filter(ImageFilter.SHARPEN)

    # 이미지 저장 (높은 품질)
    save_path = os.path.join(banner_save_folder, filename)
    image.save(save_path, quality=95, dpi=(300, 300))

# 메인 작업 로직
# 변경된 부분: 엑셀 파일 대신 단일 키워드 사용

# 폴더 생성
os.makedirs(image_final_folder, exist_ok=True)
keyword_folder = os.path.join(image_final_folder, keyword)
os.makedirs(keyword_folder, exist_ok=True)

# 배너 이미지 생성
for i in range(1, 8):
    filename = f"banner_{keyword}_{i}.png"
    generate_banner_image_with_text(keyword, font_path, keyword_folder, filename)


print("생성 완료")

생성 완료
